In [ ]:
# Task 1: ADALINE on Iris (2 features)
# Run this cell/file independently

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Utilities
def metrics_report(y_true, y_pred, pos_label=1, name="Model"):
    acc = accuracy_score(y_true, y_pred)
    cm  = confusion_matrix(y_true, y_pred, labels=[1, -1])
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", pos_label=pos_label, zero_division=0
    )
    print(f"\n=== {name} — Test Metrics ===")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}  Recall: {rec:.4f}  F1: {f1:.4f}")
    print("Confusion Matrix (rows=[+1,-1], cols=[+1,-1]):")
    print(cm)

    fig, ax = plt.subplots(figsize=(5,4))
    im = ax.imshow(cm, interpolation="nearest")
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    class_names = ["+1", "-1"]
    ax.set(
        xticks=np.arange(len(class_names)),
        yticks=np.arange(len(class_names)),
        xticklabels=class_names,
        yticklabels=class_names,
        xlabel="Predicted label",
        ylabel="True label",
        title=f"{name} — Confusion Matrix"
    )
    thresh = cm.max()/2 if cm.size else 0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], "d"),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.show()

    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "cm": cm}

def plot_histories(histories, title, ylabel="SSE"):
    plt.figure(figsize=(7,5))
    for label, values in histories.items():
        plt.plot(range(1, len(values)+1), values, label=str(label))
    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def make_iris_binary(two_features=True):
    """Create Iris binary dataset: Setosa vs Versicolor; labels -> {-1, +1}."""
    iris = load_iris()
    X = iris.data
    y = iris.target
    mask = (y == 0) | (y == 1)
    X = X[mask]
    y = y[mask]
    y = np.where(y == 1, 1, -1)

    if two_features:
        feat_idx = [2, 3]
        X = X[:, feat_idx]
        feature_names = ["petal length", "petal width"]
    else:
        feature_names = ["sepal length", "sepal width", "petal length", "petal width"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
    )
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test, feature_names

# Model
class AdalineManual:
    """Batch Gradient Descent ADALINE (identity activation)."""
    def __init__(self, lr=0.01, epochs=100, random_state=0):
        self.lr = lr
        self.epochs = epochs
        self.random_state = random_state
        self.w_ = None
        self.b_ = 0.0
        self.sse_history_ = []

    def fit(self, X, y):
        rng = np.random.default_rng(self.random_state)
        self.w_ = rng.normal(0.0, 0.01, size=X.shape[1])
        self.b_ = 0.0
        self.sse_history_.clear()

        for _ in range(self.epochs):
            net = X @ self.w_ + self.b_
            errors = y - net
            self.w_ += self.lr * (X.T @ errors) / X.shape[0]
            self.b_ += self.lr * errors.mean()
            sse = float(np.sum(errors**2))
            self.sse_history_.append(sse if np.isfinite(sse) else np.inf)
        return self

    def predict(self, X):
        return np.where(X @ self.w_ + self.b_ >= 0.0, 1, -1)

# Run
if __name__ == "__main__":
    Xtr2, Xte2, ytr2, yte2, feat2 = make_iris_binary(two_features=True)
    print(f"Two-feature Iris subset: features = {feat2}")

    adaline2 = AdalineManual(lr=0.01, epochs=100, random_state=RANDOM_STATE).fit(Xtr2, ytr2)

    plot_histories({"η=0.01": adaline2.sse_history_},
                   "ADALINE (2 features) — SSE vs Epochs", ylabel="SSE")

    yhat2 = adaline2.predict(Xte2)
    _ = metrics_report(yte2, yhat2, name="ADALINE (Iris 2 features)")
